In [1]:
import numpy as np
import sys


import collections
import re

import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, LSTM, Dropout, Input, Embedding

from keras.optimizers import RMSprop, Adam

from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras.utils import np_utils

import pandas as pd

import tensorflow as tf

Using TensorFlow backend.


In [2]:
print(tf.__version__)
print(keras.__version__)

1.3.0
2.0.8


In [3]:
# load ascii text and covert to lowercase
filename = "hotel-reviews.csv"

reviews_df = pd.read_csv(filename)

reviews_df.head()

#raw_text = open(filename).read()
#raw_text = raw_text.lower()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy


In [4]:
reviews = np.array(reviews_df['Description'])
devices = np.array(reviews_df['Device_Used'])
responses = np.array(reviews_df['Is_Response'])


for i, review in enumerate(reviews):
    review = review.lower()
    review = review.replace('""""""""""""""""""""""""""""""""', '"')
    reviews[i] = review



In [5]:
reviews_text = " ".join(reviews).lower()
len(reviews_text)

33216138

In [6]:
# create mapping of unique chars to integers

voc_cnt = collections.Counter(reviews_text.split())
vocab = sorted(voc_cnt, key=voc_cnt.get, reverse=True)


In [7]:
highFrequencyWords = [x[0] for x in voc_cnt.items() if x[1] > 10]
len(highFrequencyWords)
highFrequencyWords.insert(0,'<EOS>')



In [8]:
#print(vocab)




In [9]:


words = sorted(list(set(highFrequencyWords)))

total_words = len(reviews_text.split())

# Lookup tables
word_to_int = dict((c, i) for i, c in enumerate(highFrequencyWords))
int_to_word = dict((i, c) for i, c in enumerate(highFrequencyWords)) 

# summarize the loaded data
print(word_to_int['<EOS>'])
print(int_to_word[0])
n_vocab_words = len(words)

print("Total Vocab: ", n_vocab_words)
print("words",total_words)
print("total number of unique words", n_vocab_words)
print('total highFrequencyWords:', len(highFrequencyWords))

0
<EOS>
Total Vocab:  15634
words 6125855
total number of unique words 15634
total highFrequencyWords: 15634


In [10]:
print(int_to_word[1])
print(word_to_int['since'])
print(len(word_to_int))

the
304
15634


In [11]:
def convert_review_to_word_id(raw_review):
    review_word_ids = []
    raw_review = raw_review.lower()
    for word in raw_review.split():
        if word in word_to_int:
            word_id = word_to_int[word]
            review_word_ids.append(word_id)
        else:
            review_word_ids.append(0)
    return review_word_ids


convert_review_to_word_id("the hotel is good hahaha")

[1, 147, 69, 431, 0]

In [12]:
reviews_int = []
sentiments_int = []


for review in reviews:
    review_ids = []
    reviews_int.append(convert_review_to_word_id(review))

reviews_int = np.array(reviews_int)

for sentiment in responses:
    if sentiment == "happy":
        sentiments_int.append(1)
    else:
        sentiments_int.append(0)

sentiments_int = np_utils.to_categorical(sentiments_int)
        
print(reviews_int[3])
print(" ".join([int_to_word[word_id] for word_id in reviews_int[9]]))
print(sentiments_int[3])

[40, 142, 91, 242, 33, 243, 64, 1, 244, 76, 245, 246, 247, 153, 225, 248, 1, 249, 180, 250, 251, 18, 9, 2, 64, 1, 252, 253, 254, 252, 255, 18, 1, 212, 256, 153, 8, 9, 65, 257, 91, 9, 258, 259, 5, 1, 260, 1, 261, 33, 262, 1, 263, 0, 153, 264, 2, 265, 18, 266, 211, 153, 100, 267, 1, 268, 256, 2, 265, 269, 107, 270, 271, 272, 53, 59, 9, 268, 273, 76, 274, 275, 211, 267, 1, 276, 277, 0, 278, 83, 107, 0, 271]
wonderful staff, great location, but it was definately the price was high for a standard hotel. the free breakfast was great, actually pretty good <EOS> for a free buffet. this hotel is in the heart of it all, walking distance to everything. - things, please <EOS> coffee was bad and the white ginger line of <EOS> smells horrible!
[ 0.  1.]


In [13]:
#reviews_int = np.reshape(reviews_int, (len(reviews_int), 80, 1))
trainLen = int(len(reviews_int)*0.8)

train_dataX = reviews_int[:trainLen]
train_dataY = sentiments_int[:trainLen]
test_dataX = reviews_int[trainLen:]
test_dataY = sentiments_int[trainLen:]

print(train_dataX.shape)
print(test_dataX.shape)

from keras.preprocessing import sequence

print(train_dataX[0])

train_dataX = sequence.pad_sequences(train_dataX, maxlen=120, padding='post')
test_dataX = sequence.pad_sequences(test_dataX, maxlen=120, padding='post')

train_dataY = np.array(train_dataY)
test_dataY = np.array(test_dataY)

print(train_dataX[0])

print(train_dataX.shape)
print(train_dataY.shape)
print(test_dataX.shape)

(31145,)
(7787,)
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 5, 13, 14, 15, 16, 7, 17, 18, 9, 19, 20, 21, 22, 23, 24, 0, 25, 26, 27, 28, 21, 1, 29, 3, 30, 31, 3, 32, 33, 34, 35, 36, 37, 38]
[ 1  2  3  4  5  6  7  8  9 10 11 12  5 13 14 15 16  7 17 18  9 19 20 21 22
 23 24  0 25 26 27 28 21  1 29  3 30 31  3 32 33 34 35 36 37 38  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
(31145, 120)
(31145, 2)
(7787, 120)


In [14]:
train_dataY[2]

array([ 1.,  0.])

In [15]:
inp = Input(shape=(train_dataX.shape[1], ))
print(inp.shape)
x = Embedding(n_vocab_words, 128)(inp)
print(x.shape)
#x = Embedding(n_vocab_words, 256, input_length=80)(inp)
x = LSTM(128, return_sequences=True)(x)
x = Dropout(0.2)(x)
x = LSTM(256)(x)
x = Dropout(0.2)(x)
#x = LSTM(256)(x)
#x = Dropout(0.2)(x)
output = Dense(2, activation ='sigmoid')(x)

(?, 120)
(?, 120, 128)


In [16]:
model = Model(inputs = inp, outputs=output )
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 120)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 120, 128)          2001152   
_________________________________________________________________
lstm_1 (LSTM)                (None, 120, 128)          131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 120, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               394240    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total para

In [17]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
# define the checkpoint callback
filepath="checkpoints/hotel-sentiments-weights-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
reduce_LR = ReduceLROnPlateau(monitor='loss',factor = 0.9, patience=3,cooldown=2, min_lr = 0.00001)
callbacks_list = [checkpoint,reduce_LR]

In [25]:
model.load_weights('checkpoints/hotel-sentiments-weights-00-0.4795.hdf5')

In [19]:
model.fit(train_dataX, train_dataY, epochs=1, batch_size=64, callbacks=callbacks_list)


Epoch 1/1
31145/31145 [==============================] - 481s - loss: 0.5808 - acc: 0.7145   


In [28]:
model.save('model/hotel-sentiments-model.model')

In [29]:
model = load_model('model/hotel-sentiments-model.model')

In [34]:
print((test_dataX[7786]))
print((test_dataY[7786]))

[    1  3631    69     9  2384  7171 13926  5130   193     1    83    33
  1745    53  1979   101    76   389    18   457     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0]
[ 0.  1.]


In [35]:
print(len(test_dataX))
score, acc = model.evaluate(test_dataX[:7785], test_dataY[:7785])
print('score:', score)
print('acc:', acc)

7787
7785/7785 [==============================] - 33s    
score: 0.347085652335
acc: 0.863455362893


In [417]:
print(len(test_dataX))
score, acc = model.evaluate(test_dataX, test_dataY)
print('score:', score)
print('acc:', acc)

7787
7787/7787 [==============================] - 41s    
score: 0.411762732412
acc: 0.834275073833


In [21]:
def predictionResult(prediction):
    if np.argmax(prediction) == 0:
        return "Negative Review"
    else:
        return "Positive Review"

### Trying out the prediction...

In [30]:
def predict_review(raw_review):
    review_word_ids = convert_review_to_word_id(raw_review)
    #print(review_word_ids)
    review_word_ids = np.reshape(review_word_ids, (1, len(review_word_ids)))
    #print(review_word_ids)
    review_word_ids = sequence.pad_sequences(review_word_ids, maxlen=120, padding='post')
    #print(review_word_ids)
    review_word_ids = np.array(review_word_ids)
    review_word_ids = np.reshape(review_word_ids, (1, 120))
    return model.predict(review_word_ids)

predict_review1 = predict_review("this hotel is bad.")
predict_review2 = predict_review("this hotel is good.")
predict_review3 = predict_review("The hotel staff specially front desk are not cooperative. Their behaviour is very bad. Room are not properly clean and uun hhealthy specially bathroom is very dirty and fittings are very old.")

print('Review 1:', predict_review1, predictionResult(predict_review1))
print('Review 2:', predict_review2, predictionResult(predict_review2))
print('Review 3:', predict_review3, predictionResult(predict_review3))

Review 1: [[ 0.77551299  0.22635365]] Negative Review
Review 2: [[ 0.06128515  0.94087225]] Positive Review
Review 3: [[ 0.77552378  0.22634478]] Negative Review


In [414]:
test_idx= 7785

print("x:", test_dataX[test_idx], "y:", test_dataY[test_idx])

test_data0 = np.reshape(test_dataX[test_idx], (1, 120))
print(test_data0.shape)
model.predict(test_data0)

x: [    1   940     3    34  2957   153  1078   337    18     9   259     5
     1  1756     7   153    34   154     9   259     5     9  4686  4687
  9832     1   911  4445 11426    36    37     9   206   812   201     9
     0     1   183  2624   679     5   281    20     3     1  2334  2070
     1  1439   223     3     9     0    76   813   393   153  2114  3793
   175   281  5507    33   332     1  2863     5  1712     0   138   107
  1885   780   134  1131    76  2009   281   160    41     1   674     5
     1  2678   153   107   136   153   107    27   780    76     0    33
     0  2849    76 14888    39  2827     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0] y: [ 1.  0.]
(1, 120)


array([[ 0.2205226 ,  0.79300803]], dtype=float32)

In [208]:
print(test_dataX.shape)
print(test_dataY.shape)
print(test_dataX[0])

(7787, 120)
(7787,)
[    8  1122  2978    11     3   300     9   272     7  5189    20    13
     5     3    39   284    22   117     4   772  1330    54   469    39
   397     1   187    16 17775  1926    12     5    68    22   244     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0]
